In [1]:
import socket
import time
import numpy  as np
import matplotlib.pyplot as plt

plt.rc("text", usetex=True)

%run ./2-ImplementationFactor.ipynb
%run ./3-ImplementationPGM.ipynb
%run ./customizedLBP.ipynb

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


## Settings for Test

In [2]:
# Setting for the message
END = "END"
MSG_BREAK = "BREAK"
DILIMITER_1 = ','
DILIMITER_2 = "&"
MUL_SIGN = '*'
STATEMENT_ID_PREFIX = "S_"

In [3]:
# Helper function
def printProbDistribution(constraint, probs):
    n = probs.size
    print("-"*20)
    print(constraint + " : ")
    for i in range(0, n):
        s = '{0:0' + str(probs.ndim) + 'b}'
        indexes = [int(x) for x in list(s.format(i))]
        prob = probs
        for j in range(probs.ndim):
            prob = prob[indexes[j]]
        print(indexes, prob)
    

### Test Case 1
- Example from the paper

<img src="./img/example_fg.png" width="500"/>

In [4]:
constraint1 = "f1(x_1,x_2,x_3)"
constraint2 = "f2(x_1,x_2,x_3)"
constraint3 = "f3(x_2)"
constraint4 = "f4(x_3)"
constraint = constraint1 + constraint2 + constraint3 + constraint4
mrf = string2factor_graph(constraint)

In [5]:
# Construct probability distribution for constraint 1
prob = [0.95] * 8
prob[6] = 0.05
prob = np.array(prob)
prob = prob.reshape((2, 2, 2))
printProbDistribution(constraint1, prob)
f1 = factor(['x_1', 'x_2', 'x_3'], prob.copy())

# Construct probability distribution for constraint 2
prob = [0.95] * 8
prob[3] = 0.05
prob = np.array(prob)
prob = prob.reshape((2, 2, 2))
printProbDistribution(constraint2, prob)
f2 = factor(['x_1', 'x_2', 'x_3'], prob.copy())

# Construct probability distribution for constraint 3
prob = [0.95] * 2
prob[0] = 0.05
prob[1] = 0.95
prob = np.array(prob)
prob = prob.reshape((2))
printProbDistribution(constraint3, prob)
f3 = factor(['x_2'], prob.copy())

# Construct probability distribution for constraint 4
prob = [0.95] * 2
prob[1] = 0.05
prob[0] = 0.95
prob = np.array(prob)
prob = prob.reshape((2))
printProbDistribution(constraint4, prob)
f4 = factor(['x_3'], prob.copy())

--------------------
f1(x_1,x_2,x_3) : 
[0, 0, 0] 0.95
[0, 0, 1] 0.95
[0, 1, 0] 0.95
[0, 1, 1] 0.95
[1, 0, 0] 0.95
[1, 0, 1] 0.95
[1, 1, 0] 0.05
[1, 1, 1] 0.95
--------------------
f2(x_1,x_2,x_3) : 
[0, 0, 0] 0.95
[0, 0, 1] 0.95
[0, 1, 0] 0.95
[0, 1, 1] 0.05
[1, 0, 0] 0.95
[1, 0, 1] 0.95
[1, 1, 0] 0.95
[1, 1, 1] 0.95
--------------------
f3(x_2) : 
[0] 0.05
[1] 0.95
--------------------
f4(x_3) : 
[0] 0.95
[1] 0.05


In [6]:
mrf.change_factor_distribution('f1', f1)
mrf.change_factor_distribution('f2', f2)
mrf.change_factor_distribution('f3', f3)
mrf.change_factor_distribution('f4', f4)

In [7]:
lbp = myLBP(mrf)
var_name = 'x_1'
margProb = lbp.belief([var_name], 20)

print("LBP for ", var_name, "=", margProb[var_name])

exact = factor_marginalization(joint_distribution([f1, f2, f3, f4]), ['x_2', 'x_3']).get_distribution()
exact = exact / np.sum(exact)
print("Exact for ", var_name, "=", exact[1])

LBP for  x_1 = 0.11026699093494574
Exact for  x_1 = 0.13181818181818183
